<a href="https://colab.research.google.com/github/hank199599/data_science_from_scratch_reading_log/blob/main/Chapter13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 單純貝氏(Naive Bayes)
描述在已知一些條件下，某事件的發生機率。  
  
**建立前提**：每個事件特徵基本上是獨立的，與其他事件沒有關係。  
**特色**：不易發生過擬合



# 理論：建構垃圾郵件篩選器

## 極為簡單的版本 
利用貝氏定理計算「此郵件為垃圾郵件」的機率：
  
在「此郵件包含bitcoin這個字眼」的事件下，此郵件為垃圾郵件的機率是：
```
P(S|B)=[P(B|S)P(S)/[P(B|S)P(S)+P(B|¬S)]P(¬S)
```
* S : 此郵件為垃圾郵件的事件  
* B : 此郵件包含bitcoin這個字眼的事件
  

若手邊有大量郵件確定是垃圾郵件，亦有大量郵件確定不是垃圾郵件。  
可以輕易估算出P(B|S)以及P(B|¬S)的值。  
  
若進一步假設，一封郵件是不是垃圾郵件的機率各占一半。(即P(S)=P(¬S)=0.5)  
則原式可以被改寫為：

```
P(S|B)=[P(B|S)/[P(B|S)P(S)+P(B|¬S)]
```
假設：
* 50%的垃圾郵件含有bitcoin這個字眼
* 1%的非垃圾郵件含有bitcoin這個字眼
→確實是垃圾郵件的機率是：


```
0.5/(0.5+0.01)=98%
```



## 較為精巧的版本
**假設**：  
* 有個包含許多單詞w₁,...,wɴ的詞彙表(vocabulary)。  
* 以Xi代表「郵件中包含wi這個單詞的事件」
  
**再假設我們能估計出**：
* 垃圾郵件中包含wi的機率P(Xi|S)
* 垃圾郵件中不包含wi的機率P(Xi|¬S)
  
```
P(X₁=X₁,...,Xn=xn|S) = P(X₁=x₁|S)×...×P(Xn=xn|S)
```

假定詞彙表中只有「bitcion」與「rolex」兩個單詞，而在已知所有的垃圾郵件中：  
* 一半擁有「earn bitcoin」
* 一半擁有「authenic rolex」  

在這個情況下，垃圾郵件同時包含「bitcion」與「rolex」兩字眼的機率為0。  
若使用單純貝氏估算其機率：
```
P(X₁=1,X₂=1|S) = P(X₁=1|S)×P(X₂=1|S) = .5×.5 = .25
```
在這個假設裡，並未考慮到實際上「bitcion」與「rolex」這兩個單詞還是會互相影響。  
而非理想上的獨立事件。在忽略理想情形與現實情形有所差距的情況下，這個模型仍經常被運用在現實世界的垃圾郵件篩選器之中。
  
在實務上，通常會盡量避免很多機率值相乘。  
多個小於零的浮點數相乘後，易造成「**下溢(underflow)**」。  
根據代數關係：
* log(ab)=log(a)+log(b)
* exp(log(x))=x  
  
我們能將原式轉換為：
```
exp(log(p1)+...+log(pn))
```
假定我們手邊有大量以標示過的垃圾郵件與非垃圾郵件用於訓練。  
(log(x))=x  
  
我們能將原式轉換為：
```
exp(log(p1)+...+log(pn))
```
假定我們手邊有大量以標示過的垃圾郵件與非垃圾郵件用於訓練。  
則我們可以估算出**垃圾郵件內容中包含wi這個單詞的機率P(Xi|S)**
  
然而，在機率計算中仍會出現問題：  
假設在我們用於訓練的所有郵件中，詞彙表裡的「data」只出現在非垃圾郵件中。  
```
P("data"|S)=0
```
這代表這個分類器只要遇到任何含有「**data**」單詞的郵件會判定為**非垃圾郵件**。
即便出現「data on free bitcoin and authetic watches」這類的句子。  
  
在這種情況下，會使用一個**偽計數值k**。  
在估算垃圾郵件中出現wi這個單詞時，採取以下作法：
```
P(Xi|S) = (k+所有垃圾郵件中含有wi這個單詞的郵件數量)/(2k+所有垃圾郵件的數量)
```
```
P(Xi|¬S) = (k+所有非垃圾郵件中含有wi這個單詞的郵件數量)/(2k+所有非垃圾郵件的數量)
```
舉例來說：  
如果「data」這個單詞在98封垃圾郵件中完全沒出現，而k值設定為1時。


# 實作


## **建立物件的方式**
1. 把郵件內容依據單詞進行拆分(tokenize)
2. 把各不相同的單詞token集合起來
3. 利用re.fill把所有單詞token都提取出來
4. 用**set()**整併重複的token
 *斜體文字*
 


In [1]:
from typing import Set
import re

def tokenize(text:str) ->Set[str]:
  text = text.lower()           #轉為小寫
  all_words = re.findall("[a-z0-9]+",text) #取出其中的單詞
  return set(all_words)           #去掉重複的單詞

In [3]:
assert tokenize("Data Science is science") == {"data","science","is"}

## 針對訓練資料定義資料型別 (基本物件)
由於分類器須持續記錄訓練資料中出現的單詞token以及各種計數與標籤，因此採用物件類別的方法。

In [12]:
from typing import NamedTuple

class Message(NamedTuple):
  text:str
  is_spam:bool

## 定義處理該資料型別的方法(**method**)

* ham ：非垃圾郵件
* spam ：垃圾郵件



```python
from typing import List,Tuple,Dict,Iterable
import math
from collections import defaultdict

class NaiveBayesClassifier:
  def __init__(self,k:float=0.5)->None:
    self.k = k #平滑因子
    
    self.tokens:Set[str] = set()
    self.token_spam_counts:Dict[str,int] = defaultdict(int)
    self.token_ham_counts:Dict[str,int] = defaultdict(int)
    self.spam_messages = self.ham_messages = 0
```



### 定義**方法**：針對一堆郵件進行訓練
**方法**：針對一堆郵件進行訓練
1. 累計垃圾郵件與非垃圾郵件數量
2. 針對每封郵件進行分詞(tokenize)後的每個單詞token，根據該郵件是否是垃圾郵件來斷定該累計 **token_spam_counts** 還是 **token_ham_counts**


```python
def train(self,messages:Iterable[Message]) ->None:
  for message in messages:
    #累計郵件數量
    if message.is_spam:
      self.spam_messages += 1
    else:
      self.ham_messages += 1
    
    #累計單詞出現的次數
    for token in tokenize(message.text):
      self.tokens.add(token)
      if message.is_spam:
        self.token_spam_counts[token] += 1
      else:
        self.token_ham_counts[token] += 1

```



###  定義**方法**：預測P(spam|token)此事件的機率值
即出現某token的機率下，郵件為垃圾郵件的機率。  
  
**目標**；得到詞彙表中每個token相應的P(token|spam)與P(token|ham)  
**方法**：建立一個private輔助函式來進行計算


```python
def _probabilities(self,token:str) ->Tuple[float,float]:
  """送回P(token|spam)與P(token|ham) """
  spam = self.token_spam_counts[token]
  ham = self.token_ham_countsp[token]

  p_token_spam = (spam + self.k)/(self.spam_messages + 2 * self.k)
  p_token_ham = (ham + self.k)/(self.ham_messages + 2 * self.k)

  return p_token_spam,p_token_ham
```



### 定義**方法**：編寫預測方式
在計算最終機率值上，採用對數相加而非直接相乘。

```python
def predict(self,text:str) ->float:
  text.tokens = tokenize(text)
  log_prob_if_spam = log_prob_if_ham = 0.0

  # 以迭代方式，處理詞彙表中的每個單詞
  for token in self.tokens:
    log_prob_if_spam, p_token_ham = self._probabilities(token)

    # 如果*token*有出現在郵件中，
    # 就把「有看到該token」的對數機率值加進去
    if token in text_tokens:
      log_prob_if_spam += math.log(log_prob_if_spam)
      log_prob_if_ham += math.log(log_prob_if_ham)
    
    #否則就把「沒有看到該token」的對數機率值加進去
    #「沒有看到該token」的對數機率值就是log( 1 - 有看到該token機率值)
    else:
      log_prob_if_spam += math.log(1.0 - log_prob_if_spam)
      log_prob_if_ham += math.log(1.0 - log_prob_if_ham)
  
  prob_if_spam = math.exp(log_prob_if_spam)
  prob_if_ham = math.exp(log_prob_if_ham)
  return prob_if_spam/(prob_if_spam + log_prob_if_ham)


```



### 到此，分類器已建置完成!

In [36]:
from typing import List,Tuple,Dict,Iterable
import math
from collections import defaultdict

class NaiveBayesClassifier:
  def __init__(self,k:float=0.5)->None:
    """初始化欲包含的資料結構"""

    self.k = k #平滑因子
    self.tokens:Set[str] = set()
    self.token_spam_counts:Dict[str,int] = defaultdict(int)
    self.token_ham_counts:Dict[str,int] = defaultdict(int)
    self.spam_messages = self.ham_messages = 0

  def train(self,messages:Iterable[Message]) ->None:
    for message in messages:
      #累計郵件數量
      if message.is_spam:
        self.spam_messages += 1
      else:
        self.ham_messages += 1
      
      #累計單詞出現的次數
      for token in tokenize(message.text):
        self.tokens.add(token)
        if message.is_spam:
          self.token_spam_counts[token] += 1
        else:
          self.token_ham_counts[token] += 1
  
  def _probabilities(self,token:str) ->Tuple[float,float]:
    """預測P(spam|token)此事件的機率值
      送回P(token|spam)與P(token|ham) """
    spam = self.token_spam_counts[token]
    ham = self.token_ham_countsp[token]

    p_token_spam = (spam + self.k)/(self.spam_messages + 2 * self.k)
    p_token_ham = (ham + self.k)/(self.ham_messages + 2 * self.k)

    return p_token_spam,p_token_ham
  
  def predict(self,text:str) ->float:
    """編寫預測方法"""

    text.tokens = tokenize(text)
    log_prob_if_spam = log_prob_if_ham = 0.0

    # 以迭代方式，處理詞彙表中的每個單詞
    for token in self.tokens:
      log_prob_if_spam, p_token_ham = self._probabilities(token)

      # 如果*token*有出現在郵件中，
      # 就把「有看到該token」的對數機率值加進去
      if token in text_tokens:
        log_prob_if_spam += math.log(log_prob_if_spam)
        log_prob_if_ham += math.log(log_prob_if_ham)
      
      #否則就把「沒有看到該token」的對數機率值加進去
      #「沒有看到該token」的對數機率值就是log( 1 - 有看到該token機率值)
      else:
        log_prob_if_spam += math.log(1.0 - log_prob_if_spam)
        log_prob_if_ham += math.log(1.0 - log_prob_if_ham)
    
    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_ham = math.exp(log_prob_if_ham)

    return prob_if_spam/(prob_if_spam + log_prob_if_ham)




# 測試模型

In [37]:
#編寫一些單元測試方法，以確保模型可以正確運作
message=[Message("spam rules",is_spam=True),
     Message("ham rules",is_spam=False),
     Message("hello ham",is_spam=False)]

model = NaiveBayesClassifier(k=0.5) #假定平滑因子為0.5
model.train(message)

### 1. 檢查它是否可以正確計算出各種計數值

In [41]:
assert model.tokens == {"spam","ham","rules","hello"}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts =={"spam":1,"rules":1}
assert model.token_ham_counts == {"ham":2,"rules":1,"hello":1}


### 2.進行預測
透過手工方式進行單純貝氏邏輯的相關計算，並確認可的到相同的結果：
  
假定文字是："hello spam"

* 分析這段文字：
  * 出現"sapm"次數：1
  * 出現"ham"次數：0
  * 出現"rules"次數：0
  * 出現"hello"次數：1


In [42]:
text = "hello spam"

probs_if_spam =[
  (1+0.5)/(1+2*0.5),  #"spam"：有這個詞
  1-(0+0.5)/(1+2*0.5), #"ham"：沒有這個詞
  1-(1+0.5)/(1+2*0.5), #"rules"：沒有這個詞 
  (0+0.5)/(1+2*0.5)   #"hello"：有這個詞
]

probs_if_ham =[
  (0+0.5)/(2+2*0.5),  #"spam"：有這個詞
  1-(2+0.5)/(2+2*0.5), #"ham"：沒有這個詞
  1-(1+0.5)/(2+2*0.5), #"rules"：沒有這個詞 
  (1+0.5)/(2+2*0.5)   #"hello"：有這個詞
]

p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

# 運用模型